In [26]:
# 节点特征和标签
x = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.float)
y = torch.tensor([0, 1, 0, 1], dtype=torch.float)

In [27]:
# 边
edge_index = torch.tensor([[0, 1, 2, 0, 3],
                           [1, 0, 1, 3, 2]], dtype=torch.long)

In [30]:
# 通过创建Data对象构图
import torch
from torch_geometric.data import Data

data = Data(x=x, y=y, edge_index=edge_index)
data

Data(x=[4, 2], edge_index=[2, 5], y=[4])

## 例子

In [31]:
import pandas as pd
df = pd.read_csv('/Users/zhongshannan/Documents/data_processing/gnn.csv')
df.head()

,id,text,label,event
0,3511947309647762,震惊，转发求证：【想都不敢想 ，在美国一桶金龙鱼食用油只要8元人民币】 一桶食用油相当于中国...,real,29
1,3576100079039606,【法院无底线】湖南长沙一位小朋友上学路上捡到3万元，原地不动等失主，结果被人冒领。不知情的孩...,real,28
2,3584521306131914,"立刻检查一下你家里的牙膏，如果是黑色条马上扔掉！ 大家买膏请留心,买牙膏时注意牙膏管反面 底...",real,3
3,3869510824498067,不要再随便买奶制品给孩子喝了❗️幼儿园都发通知了家长们注意：现在得白血病的小孩越来越多，妇幼...,real,11
4,3481914981436811,【中国80吨国库黄金下落不明】香港《动向》：财政部、央行、监察部和审计署已经成立专案组，调查...,real,0


In [52]:
grouped = df.groupby('event')
for event, group in grouped:
    new_id = LabelEncoder().fit_transform(group.id)
    group = group.reset_index(drop=True)
    group['new_id'] = new_id
    node_features = group.loc[group.event == event,['new_id','text']].sort_values('new_id').text.drop_duplicates().values
    node_features = torch.LongTensor(node_features).unsqueeze(1)
    print(len(node_features))
    break

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [34]:
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from torch_geometric.data import Data
 
class gnnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(gnnDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
 
    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return ['gnn.dataset']
 
    def download(self):
        pass
    
    def process(self):
        data_list = []
        
        # 单独对一个事件进行处理
        grouped = df.groupby('event')
        for event, group in tqdm(grouped):
            # 每个事件内的每个节点id重排，从0开始计数
            new_id = LabelEncoder().fit_transform(group.id)
            group = group.reset_index(drop=True)
            group['new_id'] = new_id
            
            # 按照重排的id顺序,取出text，作为节点特征
            node_features = group.loc[group.event == event,['new_id','text']].sort_values('new_id').text.drop_duplicates().values
            # 节点特征转为LongTensor类型
            node_features = torch.LongTensor(node_features).unsqueeze(1)

            # 生成边
            target_nodes = group.new_id.values[1:]
            source_nodes = group.new_id.values[:-1]
            edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)

            # 生成data
            x = node_features
            y = torch.FloatTensor([group.label.values[0]])
            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [19]:
dataset = gnnDataset(root='/Users/zhongshannan/Documents/data_processing/')

Processing...
  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: too many dimensions 'str'

In [5]:
import torch
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops
class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='max') #  "Max" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)
 
    def message(self, x_j):
        # x_j has shape [E, in_channels]
 
        x_j = self.lin(x_j)
        x_j = self.act(x_j)
        
        return x_j
 
    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]
 
 
        new_embedding = torch.cat([aggr_out, x], dim=1)
        
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)
        
        return new_embedding

In [6]:
embed_dim = 128
from torch_geometric.nn import TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
 
        self.conv1 = SAGEConv(embed_dim, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.item_embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() +1, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.item_embedding(x)
        x = x.squeeze(1)        
 
        x = F.relu(self.conv1(x, edge_index))
 
        x, edge_index, _, batch, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
 
        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
 
        x = F.relu(self.conv3(x, edge_index))
 
        x, edge_index, _, batch, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
 
        x = x1 + x2 + x3
 
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)
 
        x = torch.sigmoid(self.lin3(x)).squeeze(1)
 
        return x

In [ ]:
from torch.utils.data import DataLoader

def train():
    model.train()
    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)
    
device = torch.device('cuda')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.BCELoss()
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
for epoch in range(10):
    train()